### atmacup10_EDA

In [112]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from PIL import ImageColor
import colorsys
from geopy.geocoders import Nominatim
pd.set_option('display.max_Columns', 100)

### sample_submission.csv

In [6]:
sample_sub = pd.read_csv('../input/atmacup10__sample_submission.csv')
print('sample_sub', sample_sub.shape)
sample_sub.head()

sample_sub (12008, 1)


,likes
0,68.500677
1,9.184876
2,22.090073
3,68.801811
4,58.470058


### color.csv

In [12]:
color = pd.read_csv('../input/color.csv')
color_rgb = pd.DataFrame(color['hex'].str.strip().map(ImageColor.getrgb).values.tolist(), columns=['R', 'G', 'B'])
color = pd.concat([color, color_rgb], axis=1)
print('color', color.shape)
color.head()

color (73636, 6)


,object_id,percentage,hex,R,G,B
0,001020bd00b149970f78,0,#C7C7B6,199,199,182
1,001020bd00b149970f78,9,#5E5333,94,83,51
2,001020bd00b149970f78,14,#3E321B,62,50,27
3,001020bd00b149970f78,3,#B8AF84,184,175,132
4,001020bd00b149970f78,62,#15120C,21,18,12


In [106]:
def rgb_to_hsv(r, g, b):
    return colorsys.rgb_to_hsv(r/255, g/255, b/255)

def rgb_to_h(row):
    return rgb_to_hsv(row['R'], row['G'], row['B'])[0]

def rgb_to_s(row):
    return rgb_to_hsv(row['R'], row['G'], row['B'])[1]

def rgb_to_v(row):
    return rgb_to_hsv(row['R'], row['G'], row['B'])[2]

In [107]:
color['h'] = color.apply(rgb_to_h, axis=1)
color['s'] = color.apply(rgb_to_s, axis=1)
color['v'] = color.apply(rgb_to_v, axis=1)

In [108]:
color

,object_id,percentage,hex,R,G,B,h,s,v
0,001020bd00b149970f78,0,#C7C7B6,199,199,182,0.166667,0.085427,0.780392
1,001020bd00b149970f78,9,#5E5333,94,83,51,0.124031,0.457447,0.368627
2,001020bd00b149970f78,14,#3E321B,62,50,27,0.109524,0.564516,0.243137
3,001020bd00b149970f78,3,#B8AF84,184,175,132,0.137821,0.282609,0.721569
4,001020bd00b149970f78,62,#15120C,21,18,12,0.111111,0.428571,0.082353
...,...,...,...,...,...,...,...,...,...
73631,fff1d87d79953ddab2c6,11,#9D9F91,157,159,145,0.190476,0.088050,0.623529
73632,fff1d87d79953ddab2c6,3,#595338,89,83,56,0.136364,0.370787,0.349020
73633,fff1d87d79953ddab2c6,16,#6E7669,110,118,105,0.269231,0.110169,0.462745
73634,fff1d87d79953ddab2c6,60,#3A1F11,58,31,17,0.056911,0.706897,0.227451


### palette.csv

In [13]:
palette = pd.read_csv('../input/palette.csv')
print('palette', palette.shape)
palette.head()

palette (527890, 5)


,ratio,color_r,color_g,color_b,object_id
0,0.013781,40,4,0,000405d9a5e3f49fc49d
1,0.040509,221,189,129,000405d9a5e3f49fc49d
2,0.036344,207,175,117,000405d9a5e3f49fc49d
3,0.033316,230,197,129,000405d9a5e3f49fc49d
4,0.039600,194,161,106,000405d9a5e3f49fc49d


### historical_person.csv

In [18]:
historical_person = pd.read_csv('../input/historical_person.csv')
print('historical_person', historical_person.shape)
historical_person.head()

historical_person (6745, 2)


,object_id,name
0,00133be3ff222c9b74b0,Staten-Generaal
1,00133be3ff222c9b74b0,"Álvarez de Toledo, Fernando (3e hertog van Alva)"
2,0026e030a0209b834b3e,Raad van Beroerten
3,0026e030a0209b834b3e,"Álvarez de Toledo, Fernando (3e hertog van Alva)"
4,00440ec5a4783b4b6bdb,Dolle Mina


In [87]:
historical_person_unique = pd.DataFrame(historical_person['name'].value_counts())
historical_person_unique.head(10)

,name
"Willem III (prins van Oranje en koning van Engeland, Schotland en Ierland)",193
Verenigde Oostindische Compagnie,134
Lodewijk XIV (koning van Frankrijk),134
Willem I (prins van Oranje),95
Wilhelm II (keizer van Duitsland),84
"Álvarez de Toledo, Fernando (3e hertog van Alva)",83
Wilhelmina (koningin der Nederlanden),81
"Wirix, F.J.",67
"Djatiroto, Suikeronderneming",67
Handels Vereeniging Amsterdam,67


### material.csv

In [19]:
material = pd.read_csv('../input/material.csv')
print('material', material.shape)
material.head()

material (35394, 2)


,object_id,name
0,000405d9a5e3f49fc49d,photographic paper
1,000405d9a5e3f49fc49d,cardboard
2,001020bd00b149970f78,oil paint (paint)
3,001020bd00b149970f78,panel
4,0011d6be41612ec9eae3,oil paint (paint)


In [83]:
def get_one_hot_encoding(df_input, df_merge, prefix):
    value_counts = df_merge['name'].value_counts()
    col_list = list(value_counts.index[0:9])
    dummy = pd.get_dummies(df_merge['name'])[col_list]
    dummy = pd.concat([df_merge, dummy], axis=1)
    group = dummy.groupby('object_id').sum()
    group = group.add_prefix(prefix).reset_index()
    out = pd.merge(df_input[['object_id']], group, on='object_id', how='left')
    return out.fillna(0).drop(columns=['object_id'])

In [84]:
get_one_hot_encoding(train, material, 'MT_')

,MT_paper,MT_photographic paper,MT_oil paint (paint),MT_cardboard,MT_canvas,MT_panel,MT_glass,MT_copper (metal),MT_baryta paper
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
12021,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12022,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12023,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12024,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
material_unique = pd.DataFrame(material['name'].value_counts())
material_unique.head(10)

,name
paper,13251
photographic paper,7615
oil paint (paint),4131
cardboard,4074
canvas,2220
panel,1695
glass,756
copper (metal),355
baryta paper,208
velvet (fabric weave),149


### object_collection.csv

In [25]:
object_collection = pd.read_csv('../input/object_collection.csv')
print('object_collection', object_collection.shape)
object_collection.head()

object_collection (14210, 2)


,object_id,name
0,001020bd00b149970f78,paintings
1,0011d6be41612ec9eae3,paintings
2,00133be3ff222c9b74b0,prints
3,00181d86ff1a7b95864e,paintings
4,001c52ae28ec106d9cd5,paintings


In [85]:
get_one_hot_encoding(train, object_collection, 'OC_')

,OC_prints,OC_paintings,OC_paper,OC_Navy Model Room,OC_drawings,OC_dollhouse,OC_musical instruments,OC_lace,OC_jewellery
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
12021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12022,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12023,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12024,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
object_collection_unique = pd.DataFrame(object_collection['name'].value_counts())
object_collection_unique.head(10)

,name
prints,9905
paintings,4200
paper,32
Navy Model Room,29
drawings,22
dollhouse,13
musical instruments,4
lace,3
jewellery,1
glass,1


### production_place.csv

In [117]:
production_place = pd.read_csv('../input/production_place.csv')
print('production_place', production_place.shape)
production_place.head()

production_place (17686, 2)


,object_id,name
0,0012765f7a97ccc3e9e9,Netherlands
1,00133be3ff222c9b74b0,Amsterdam
2,00133be3ff222c9b74b0,Northern Netherlands
3,0017be8caa87206532cb,Amsterdam
4,001b2b8c9d3aa1534dfe,Suriname


In [43]:
production_place_unique = pd.DataFrame(production_place['name'].value_counts())
production_place_unique.head(10)

,name
Amsterdam,4270
Netherlands,2206
Northern Netherlands,1160
Antwerp,1082
Paris,947
France,668
The Hague,495
Haarlem,438
unknown,393
Cologne,346


In [121]:
def place2country(address):
    geolocator = Nominatim(user_agent='sample', timeout=200)
    loc = geolocator.geocode(address, language='en')
    coordinates = (loc.latitude, loc.longitude)
    location = geolocator.reverse(coordinates, language='en')
    country = location.raw['address']['country']
    return country

def get_country_dict():
    place_list = production_place['name'].unique()
    country_dict = {}
    for place in place_list:
        try:
            country = place2country(place)
            country_dict[place] = country
        except:
            # 国名を取得できない場合はnan
            print(place)
            country_dict[place] = np.nan
    return country_dict

# country_dict = get_country_dict()

# production_country = production_place.copy()
# production_country['country_name'] = production_country['name'].map(country_dict)
# production_country['name'] = production_country['country_name']
# production_country.drop(columns=['country_name'], inplace=True)
# production_country.to_csv('../input/production_country.csv', index=False)

In [125]:
production_country = pd.read_csv('../input/production_country.csv')
print('production_country', production_country.shape)
production_country_unique = pd.DataFrame(production_country['name'].value_counts())
production_country_unique.head(17)

production_country (17686, 2)


,name
Netherlands,10125
France,1704
Belgium,1411
Indonesia,987
Italy,880
Germany,784
United Kingdom,503
Denmark,289
Suriname,282
United States,239


### technique.csv

In [33]:
technique = pd.read_csv('../input/technique.csv')
print('technique', technique.shape)
technique.head()

technique (20481, 2)


,object_id,name
0,000405d9a5e3f49fc49d,albumen print
1,0012765f7a97ccc3e9e9,salted paper print
2,0012765f7a97ccc3e9e9,albumen print
3,00133be3ff222c9b74b0,etching
4,0017be8caa87206532cb,albumen print


In [42]:
technique_unique = pd.DataFrame(technique['name'].value_counts())
technique_unique.head(10)

,name
etching,5936
engraving,5563
albumen print,5544
gelatin silver print,1158
letterpress printing,722
drypoint,229
salted paper print,221
slide,203
painting,184
steel engraving,161


### maker.csv

In [37]:
maker = pd.read_csv('../input/maker.csv')
print('maker', maker.shape)
maker.head()

maker (3291, 6)


,name,place_of_birth,date_of_birth,date_of_death,place_of_death,nationality
0,'t Hooft,NaN,NaN,NaN,NaN,NaN
1,A. & G. Taylor,NaN,NaN,NaN,NaN,NaN
2,A. Böeseken,'s-Heerenberg,1834-01-12,1919-02-11,Rotterdam,NaN
3,A. Cucumus & Cie.,NaN,NaN,NaN,NaN,NaN
4,A. Dandoy,NaN,NaN,NaN,NaN,NaN


In [41]:
maker_place_of_birth_unique = pd.DataFrame(maker['place_of_birth'].value_counts())
maker_place_of_birth_unique.head(10)

,place_of_birth
Amsterdam,354
Antwerpen,134
Den Haag,127
Parijs,83
Haarlem,82
Utrecht,69
Leiden,56
Rotterdam,53
Dordrecht,43
Brussel,34


### principal_maker.csv

In [46]:
principal_maker = pd.read_csv('../input/principal_maker.csv')
print('principal_maker', principal_maker.shape)
principal_maker.head()

principal_maker (24538, 6)


,id,object_id,qualification,roles,productionPlaces,maker_name
0,1,000405d9a5e3f49fc49d,mentioned on object,NaN,NaN,Bernardus Bruining
1,2,001020bd00b149970f78,workshop of,painter,NaN,Jan Antonisz van Ravesteyn
2,3,0011d6be41612ec9eae3,NaN,painter,NaN,Jan Hackaert
3,4,0012765f7a97ccc3e9e9,attributed to,NaN,Netherlands,Richard Tepe
4,5,00133be3ff222c9b74b0,NaN,print maker,Northern Netherlands,Theodoor Koning


In [48]:
principal_maker_qualification_unique = pd.DataFrame(principal_maker['qualification'].value_counts())
principal_maker_qualification_unique.head(10)

,qualification
mentioned on object,7889
attributed to,1891
possibly,714
workshop of,676
signed by artist,365
circle of,189
copy after,133
manner of,42
rejected attribution,36
follower of,29


In [49]:
principal_maker_name_unique = pd.DataFrame(principal_maker['maker_name'].value_counts())
principal_maker_name_unique.head(10)

,maker_name
anonymous,3470
Johann Sadeler (I),661
Bernard Picart,541
Richard Tepe,364
anoniem (Monumentenzorg),359
Woodbury & Page,294
Jan Luyken,284
Raphaël Sadeler (I),267
Romeyn de Hooghe,267
A.J.M. Mulder,260


### principal_maker_occupation

In [51]:
principal_maker_occupation = pd.read_csv('../input/principal_maker_occupation.csv')
print('principal_maker_occupation', principal_maker_occupation.shape)
principal_maker_occupation.head()

principal_maker_occupation (25372, 2)


,id,name
0,2,draughtsman
1,2,painter
2,3,print maker
3,3,draughtsman
4,3,painter


In [52]:
principal_maker_occupation_name_unique = pd.DataFrame(principal_maker_occupation['name'].value_counts())
principal_maker_occupation_name_unique.head(10)

,name
print maker,10758
draughtsman,7151
painter,5101
publisher,1743
goldsmith,268
printer,186
sculptor,105
designer,54
bookseller,3
auctioneer,1


### train

In [55]:
train = pd.read_csv('../input/train.csv')
print('train', train.shape)
train.head().T

train (12026, 19)


,0,1,2,3,4
object_id,0011d6be41612ec9eae3,0012765f7a97ccc3e9e9,0017be8caa87206532cb,00181d86ff1a7b95864e,001c52ae28ec106d9cd5
art_series_id,93c092ba70beab248f31,95c14fb11c54281ad7e0,4c406890d208fe01f8fb,fa0891535500a4973db2,8065ed5063c5887e677d
title,The Avenue of Birches,Struik in bloei,Portret van een vrouw,A St Bernard Dog,Woelige zee met zeilschepen
description,NaN,NaN,"Portret van eenvrouw, zittend naast een tafel.","Een sint-bernardshond, staand in een landschap...",Woelige zee met zeilschepen.
long_title,"The Avenue of Birches, Jan Hackaert, 1660 - 1685","Struik in bloei, Richard Tepe (attributed to),...","Portret van een vrouw, Tresling & Comp., 1860 ...","A St Bernard Dog, Bernard te Gempt, c. 1850 - ...","Woelige zee met zeilschepen, anonymous, 1825 -..."
principal_maker,Jan Hackaert,Richard Tepe,Tresling & Comp.,Bernard te Gempt,anonymous
principal_or_first_maker,Jan Hackaert,Richard Tepe,Tresling & Comp.,Bernard te Gempt,anonymous
sub_title,h 66.5cm × w 53.7cm × t 2.5cm × d 4.7cm,h 165mm × w 223mm,h 87mm × w 56mm,h 179cm × w 248cm × t 4cm,h 13cm × w 17.5cm × d 0.7cm
copyright_holder,NaN,erven Richard Tepe,NaN,NaN,NaN
more_title,The Avenue of Birches,Struik in bloei,Portret van een vrouw,A St Bernard Dog,Woelige zee met zeilschepen


In [57]:
train.dtypes

object_id                    object
art_series_id                object
title                        object
description                  object
long_title                   object
principal_maker              object
principal_or_first_maker     object
sub_title                    object
copyright_holder             object
more_title                   object
acquisition_method           object
acquisition_date             object
acquisition_credit_line      object
dating_presenting_date       object
dating_sorting_date         float64
dating_period                 int64
dating_year_early           float64
dating_year_late            float64
likes                         int64
dtype: object

### test

In [56]:
test = pd.read_csv('../input/test.csv')
print('test', test.shape)
test.head().T

test (12008, 18)


,0,1,2,3,4
object_id,000405d9a5e3f49fc49d,001020bd00b149970f78,00133be3ff222c9b74b0,001b2b8c9d3aa1534dfe,00220cd4bfa082d2aa20
art_series_id,a2aabd3cc8859f4260ad,5d6a286cc3ab64c2c043,e2cbc48624f787d2edd2,2435b200cca8b2f3ce3a,aa03c5a95d229166982f
title,Twee personen voor de Duizendjarige den te Wol...,"Portrait of Philipp Ernst, Count of Hohenlohe ...","Alva eist de Tiende Penning op, 1569","Groepen uit ""Zes meisjes en geen man"" ten Gouv...",Christus op een voetstuk met rondom zijn voorv...
description,NaN,"Portret van Philip Ernst (1585-1629), graaf va...",De hertog van Alva eist in een vergadering van...,"Scène uit de opvoering getiteld ""Zes meisjes e...",In het midden staat Christus op een voetstuk. ...
long_title,Twee personen voor de Duizendjarige den te Wol...,"Portrait of Philipp Ernst, Count of Hohenlohe ...","Alva eist de Tiende Penning op, 1569, Theodoor...","Groepen uit ""Zes meisjes en geen man"" ten Gouv...",Christus op een voetstuk met rondom zijn voorv...
principal_maker,Bernardus Bruining,Jan Antonisz van Ravesteyn,Theodoor Koning,Théodore van Lelyveld,Jan Luyken
principal_or_first_maker,Bernardus Bruining,Jan Antonisz van Ravesteyn,Theodoor Koning,Théodore van Lelyveld,Jan Luyken
sub_title,h 167mm × w 108mm,h 29.7cm × w 24.1cm,h 270mm × w 215mm,h 116mm × w 159mm,h 108mm × w 184mm
copyright_holder,NaN,NaN,NaN,erven Théodore van Lelyveld,NaN
more_title,Twee personen voor de Duizendjarige den te Wol...,"Portrait of Philipp Ernst, Count of Hohenlohe ...","Alva eist de Tiende Penning op, 1569 Vordering...","Groepen uit ""Zes meisjes en geen man"" ten Gouv...",Christus op een voetstuk met rondom zijn voorv...
